In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
!pip install transformers datasets peft wandb sacremoses

In [52]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from peft import LoraConfig, get_peft_model

In [53]:
# Check GPU availability
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Please enable it in runtime settings.")

GPU is available!
GPU name: NVIDIA A100-SXM4-40GB


In [54]:
import torch
torch.cuda.empty_cache()

# Load Dataset

In [55]:
# File paths
training_set = "/content/drive/MyDrive/P2/T1/Dataset/non_overlapped/LLM-no-train.csv"
validation_set = "/content/drive/MyDrive/P2/T1/Dataset/non_overlapped/LLM-no-val.csv"
testing_set = "/content/drive/MyDrive/P2/T1/Dataset/non_overlapped/LLM-no-test.csv"

# Load datasets
train_data = pd.read_csv(training_set)
val_data = pd.read_csv(validation_set)
test_data = pd.read_csv(testing_set)

# Add prompt column to the datasets
for dataset in [train_data, val_data, test_data]:
    dataset['prompt'] = dataset.apply(
        lambda row: f"Symptoms: {row['symptoms']}\nPredict the disease:",
        axis=1
    )

# Create mappings for text-based labels
unique_labels = train_data['disease_label'].unique()
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for idx, label in enumerate(unique_labels)}

# Map text labels to numeric indices
for dataset in [train_data, val_data, test_data]:
    dataset['target'] = dataset['disease_label'].map(label_to_id)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

disease_counts = train_data['disease_label'].value_counts()
print("Disease Distribution in Training Data:")
print(disease_counts)


Number of training samples: 37384
Number of validation samples: 7974
Number of testing samples: 7974
Disease Distribution in Training Data:
disease_label
infection urinary tract               762
fibroid tumor                         761
carcinoma breast                      758
hyperglycemia                         757
ulcer peptic                          757
cirrhosis                             757
schizophrenia                         756
effusion pericardial                  753
deep vein thrombosis                  751
adhesion                              751
obesity                               751
obesity morbid                        749
neutropenia                           749
epilepsy                              747
overload fluid                        746
adenocarcinoma                        746
hepatitis                             746
dependence                            746
tachycardia sinus                     745
manic disorder                        744
gastri

In [56]:
disease_counts = train_data['disease_label'].value_counts()
print("Disease Distribution in Training Data:")
print(disease_counts)

Disease Distribution in Training Data:
disease_label
infection urinary tract               762
fibroid tumor                         761
carcinoma breast                      758
hyperglycemia                         757
ulcer peptic                          757
cirrhosis                             757
schizophrenia                         756
effusion pericardial                  753
deep vein thrombosis                  751
adhesion                              751
obesity                               751
obesity morbid                        749
neutropenia                           749
epilepsy                              747
overload fluid                        746
adenocarcinoma                        746
hepatitis                             746
dependence                            746
tachycardia sinus                     745
manic disorder                        744
gastritis                             743
delirium                              740
infection              

# Load LLM

In [57]:
# Import the required classes
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load ClinicalBERT model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # ClinicalBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels)
)

print("ClinicalBERT model and tokenizer loaded successfully!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ClinicalBERT model and tokenizer loaded successfully!


# LoRA (Low-Rank Adaptation)

In [58]:

# Configure LoRA for BioBERT
lora_config = LoraConfig(
    r=8,  # Low-rank adaptation rank
    lora_alpha=32,  # Scaling factor
    target_modules=["query", "value"],  # LoRA applied to attention layers
    lora_dropout=0.1,  # Dropout rate for regularization
    task_type="SEQ_CLS"  # Sequence classification task
)

model = get_peft_model(model, lora_config)
print("LoRA applied successfully to BioBERT!")


LoRA applied successfully to BioBERT!


# Tokenization function

In [59]:
# Tokenization function for the dataset
def tokenize_function(examples):
    # Tokenize the 'prompt' column
    tokenized_inputs = tokenizer(
        examples['prompt'],  # Input text (symptoms prompt)
        truncation=True,
        padding='max_length',
        max_length=256,  # Adjust as needed
        return_tensors="pt"
    )
    # Add 'labels' field (numeric targets)
    tokenized_inputs["labels"] = examples["target"]
    return tokenized_inputs

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the dataset format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/37384 [00:00<?, ? examples/s]

Map:   0%|          | 0/7974 [00:00<?, ? examples/s]

Map:   0%|          | 0/7974 [00:00<?, ? examples/s]

# Training Arguments

In [60]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.005,
    logging_dir="./logs",
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False  # Disable column filtering
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset= val_dataset,
    tokenizer=tokenizer
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-60-0a87ca8a542b>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [61]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset= val_dataset,
    tokenizer=tokenizer
)

<ipython-input-61-d09d09b8867b>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [62]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.960400,1.895054
2,1.023900,0.901717
3,0.522300,0.489130
4,0.369800,0.303497
5,0.344900,0.224935
6,0.234900,0.206074


TrainOutput(global_step=14022, training_loss=1.0587119381253651, metrics={'train_runtime': 1841.4274, 'train_samples_per_second': 121.81, 'train_steps_per_second': 7.615, 'total_flos': 2.9640770928820224e+16, 'train_loss': 1.0587119381253651, 'epoch': 6.0})

# Training and Validation Loss Graph

In [63]:
"""
import matplotlib.pyplot as plt

# Data from the table
epochs = [1, 2, 3, 4, 5, 6]
training_loss = [2.616900, 1.970200, 1.714200, 1.640900, 1.364900, 1.114800]
validation_loss = [2.713036, 1.945025, 1.528872, 1.351326, 1.194266, 1.159532]

# Plot the graph
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, label="Training Loss", marker="o")
plt.plot(epochs, validation_loss, label="Validation Loss", marker="o")

# Add labels, title, and legend
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()
"""

'\nimport matplotlib.pyplot as plt\n\n# Data from the table\nepochs = [1, 2, 3, 4, 5, 6]\ntraining_loss = [2.616900, 1.970200, 1.714200, 1.640900, 1.364900, 1.114800]\nvalidation_loss = [2.713036, 1.945025, 1.528872, 1.351326, 1.194266, 1.159532]\n\n# Plot the graph\nplt.figure(figsize=(10, 6))\nplt.plot(epochs, training_loss, label="Training Loss", marker="o")\nplt.plot(epochs, validation_loss, label="Validation Loss", marker="o")\n\n# Add labels, title, and legend\nplt.xlabel("Epoch")\nplt.ylabel("Loss")\nplt.title("Training and Validation Loss Over Epochs")\nplt.legend()\nplt.grid(True)\n\n# Show the plot\nplt.show()\n'

# Evaluate the fine-tuned model directly

In [70]:
from datasets import Dataset

# Load the test dataset
test_file_path = '/content/drive/MyDrive/P2/T1/Dataset/non_overlapped/LLM-no-test.csv'
test_data = pd.read_csv(test_file_path)

# Add prompt column (if not already present)
test_data['prompt'] = test_data.apply(
    lambda row: f"Symptoms: {row['symptoms']}\nPredict the disease:",
    axis=1
)

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_data)

In [71]:
import re

def predict_disease(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256).to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=-1).item()
        return id_to_label[predicted_class_id]  # Map numeric index back to text label

# Test Predictions
test_data['predicted_disease'] = test_data['prompt'].apply(predict_disease)

# Print Predictions
for index, row in test_data.iterrows():
    print(f"Prompt: {row['prompt']}")
    print(f"Predicted Disease: {row['predicted_disease']}")


Streaming output truncated to the last 5000 lines.
Predict the disease:
Predicted Disease: overload fluid
Prompt: Symptoms: Patient shows symptoms as follows: have stool color yellow; have pain abdominal; have apyrexial; have sore to touch.
Predict the disease:
Predicted Disease: pancreatitis
Prompt: Symptoms: Patient shows symptoms as follows: have stool color yellow; have apyrexial; have sore to touch; have diarrhea; have pain.
Predict the disease:
Predicted Disease: pancreatitis
Prompt: Symptoms: Patient shows symptoms as follows: have pain abdominal; have sore to touch; have diarrhea; have rigor - temperature-associated observation; have pain.
Predict the disease:
Predicted Disease: pancreatitis
Prompt: Symptoms: Patient shows symptoms as follows: have stool color yellow; have sore to touch; have rigor - temperature-associated observation.
Predict the disease:
Predicted Disease: pancreatitis
Prompt: Symptoms: Patient shows symptoms as follows: have pain abdominal; have apyrexial; h

# Save fine-tuned Model

In [66]:
"""
# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt')
tokenizer.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt')
print("Fine-tuned BioGPT model and tokenizer saved successfully!")
"""

'\n# Save the fine-tuned model\nmodel.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt\')\ntokenizer.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt\')\nprint("Fine-tuned BioGPT model and tokenizer saved successfully!")\n'

# Generate Classification Report

In [72]:
from sklearn.metrics import classification_report
import pandas as pd

# Assume `test_data` is your test dataset after generating predictions
# The test_data dataframe should already have columns: 'Label' (true labels) and 'Predicted' (model predictions)

# True labels and predicted labels
true_labels = test_data['disease_label']
predicted_labels = test_data['predicted_disease']

# Generate classification report
report = classification_report(true_labels, predicted_labels, output_dict=False)
print(report)

# Save classification report to a file
report_dict = classification_report(true_labels, predicted_labels, output_dict=True)
pd.DataFrame(report_dict).transpose().to_csv('/content/drive/MyDrive/P2/T1/Dataset/non_overlapped/LLM/BioBERT-no-class.csv', index=True)
print("Classification report saved to '/content/drive/MyDrive/P2/T1/Dataset/non_overlapped/LLM/BioBERT-no-class.csv'")


                                    precision    recall  f1-score   support

                    adenocarcinoma       0.97      1.00      0.98       159
                          adhesion       0.97      0.96      0.96       160
                     affect labile       1.00      0.96      0.98        75
                         arthritis       0.98      0.95      0.97       123
                            asthma       0.88      0.94      0.91        71
      benign prostatic hypertrophy       1.00      0.89      0.94        75
                  biliary calculus       0.97      0.85      0.91       146
                  carcinoma breast       0.99      0.99      0.99       162
chronic obstructive airway disease       0.97      0.97      0.97       119
                         cirrhosis       0.99      0.94      0.97       162
                           colitis       0.95      0.98      0.97       150
         coronary arteriosclerosis       0.97      0.84      0.90       106
           

In [68]:
"""
output_path = '/content/drive/MyDrive/P2/LLM/2BioBERT_Predictions.csv'
test_data.to_csv(output_path, index=False)
print(f"Predictions saved to: {output_path}")
"""

'\noutput_path = \'/content/drive/MyDrive/P2/LLM/2BioBERT_Predictions.csv\'\ntest_data.to_csv(output_path, index=False)\nprint(f"Predictions saved to: {output_path}")\n'